In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm # 模型拟合用的库


In [11]:
df_close = pd.read_csv("history_close.csv").set_index(['trade_date'])
df_train_ret = df_close.pct_change().iloc[1:]
df_train_ret # 训练用的数据格式

,600000.SH,600009.SH,600016.SH,600028.SH,600030.SH,600031.SH,600036.SH,600048.SH,600050.SH,600104.SH,...,601688.SH,601818.SH,601857.SH,601888.SH,601899.SH,601995.SH,603259.SH,603288.SH,603501.SH,603986.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20201103,0.005405,0.017419,0.005848,0.010230,0.023148,-0.035496,0.024596,0.005165,0.023355,-0.012033,...,0.013416,0.025773,0.014778,0.017284,0.063978,0.100000,-0.003506,-0.005200,-0.010238,-0.011187
20201104,0.004301,0.002023,-0.001938,-0.005063,0.000000,0.016225,0.040980,-0.001927,-0.008299,0.067199,...,-0.020367,0.010050,-0.002427,0.023614,0.002614,0.068242,0.011873,0.009085,-0.021290,0.008362
20201105,0.002141,0.041317,0.003883,0.007634,0.016011,0.041277,-0.003261,0.024453,0.018828,0.082251,...,0.007796,0.007463,0.002433,-0.011510,-0.001304,0.065463,0.016080,0.040022,0.014218,0.008780
20201106,0.003205,-0.001193,0.000000,0.005051,0.001713,0.028048,-0.007245,0.015075,0.002053,0.008000,...,-0.008767,0.002469,0.002427,-0.018891,0.031332,0.100000,-0.004192,0.007412,0.024813,0.025919
20201109,0.004260,0.048536,0.009671,0.007538,0.035226,0.001819,0.012947,0.001856,0.010246,0.024531,...,0.038502,0.004926,0.012107,0.016198,0.029114,0.099961,0.026115,-0.006062,0.091833,0.060945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230313,-0.012587,0.015117,0.000000,0.061420,0.007004,0.008436,0.000571,-0.011896,0.054007,-0.010519,...,0.004029,0.009901,0.044118,0.027954,0.062937,0.011477,-0.015582,0.018738,0.011362,0.004652
20230314,-0.005666,-0.007273,-0.005882,-0.001808,-0.013413,-0.027886,-0.013409,0.002125,0.021488,-0.004252,...,-0.017657,-0.006536,-0.022887,-0.002462,-0.010691,-0.021954,0.000633,-0.010436,0.016912,0.034877
20230315,0.008547,-0.003140,0.002959,0.001812,0.009567,0.006885,0.008676,0.013428,-0.029126,0.004270,...,0.006536,0.013158,0.018018,-0.018245,-0.017456,-0.003783,-0.009112,-0.010018,0.000238,-0.000762


In [37]:
# 训练
component = 1
df_ret_std = (df_train_ret - df_train_ret.mean())/df_train_ret.std()
eigenvals_P, eigenvecs_P = np.linalg.eig(df_ret_std.corr())
idx_P = eigenvals_P.argsort()[::-1]  
pca_eigenvecs = eigenvecs_P[:, idx_P][:, :component] 
fct_P = np.dot(df_train_ret,pca_eigenvecs)
OLSmodels_P = {stock: sm.OLS(df_train_ret[stock], fct_P).fit() for stock in df_train_ret.columns}
beta_P = pd.DataFrame({stock: model_P.params for stock, model_P in OLSmodels_P.items()})

# 每分钟输入一次新的df_cur_ret
df_cur_ret = df_train_ret.iloc[-1] # 先用df_train_ret.iloc[-1]替代
cur_res = df_cur_ret - df_cur_ret.dot(np.array(pca_eigenvecs)).dot(beta_P)
long = cur_res[cur_res < 0].sort_values()
buy = long / long.sum()
buy = buy[buy > 0.05]
buy.to_dict() # 资产配置比例


{'600588.SH': 0.24244616673454708,
 '600050.SH': 0.23863067654133177,
 '600570.SH': 0.09817050031521839,
 '600837.SH': 0.08043809365014444}